In [18]:
import pennylane as qml
from pennylane import numpy as np

In [19]:
N = 2
data_wires = range(0, 2**N)
tmp_data_wires = range(2**N, 2**N + 2**(N-1))
print("data_wires = ", data_wires)
index_wires = range(2**N + 2**(N-1), 2**N + 2**(N-1) + N)
incident_wire = range(2**N + 2**(N-1) + N, 2**N + 2**(N-1) + N + 1)
tot_wires = range(0, 2**N + 2**(N-1) + N + 1)

dev = qml.device("default.qubit", wires=tot_wires, shots=1)

data_wires =  range(0, 4)


In [47]:
def traverse():
    qml.PauliX(index_wires[0])
    qml.CSWAP(wires=[index_wires[0], incident_wire[0], tmp_data_wires[0]])
    qml.PauliX(index_wires[0])
    qml.CSWAP(wires=[index_wires[0], incident_wire[0], tmp_data_wires[1]])

    qml.PauliX(index_wires[1])
    qml.CSWAP(wires=[index_wires[1], tmp_data_wires[0], data_wires[0]])
    qml.PauliX(index_wires[1])
    qml.CSWAP(wires=[index_wires[1], tmp_data_wires[0], data_wires[1]])

    qml.PauliX(index_wires[1])
    qml.CSWAP(wires=[index_wires[1], tmp_data_wires[1], data_wires[2]])
    qml.PauliX(index_wires[1])
    qml.CSWAP(wires=[index_wires[1], tmp_data_wires[1], data_wires[3]])
    return


@qml.qnode(dev)
def QRAM(x, idx):    
    qml.PauliX(wires=incident_wire[0])
    qml.BasisEmbedding(idx, wires=index_wires)

    qml.BasisEmbedding(x, wires=data_wires)

    traverse()
    qml.CNOT(wires=[incident_wire[0], incident_wire[0] + 1])
    qml.adjoint(traverse)()

    # qml.PauliX(index_wires[1])
    # qml.CSWAP(wires=[index_wires[1], incident_wire[0], data_wires[0]])
    # qml.PauliX(index_wires[1])
    # qml.CSWAP(wires=[index_wires[1], incident_wire[0], data_wires[1]])


    # qml.CSWAP(wires=[index_wires[0], data_wires[0], data_wires[1]])

    # qml.PauliX(index_wires[0])
    # qml.CSWAP(wires=[index_wires[0], data_wires[1], data_wires[2]])
    # qml.PauliX(index_wires[0])

    # qml.CSWAP(wires=[index_wires[0], data_wires[1], data_wires[3]])

    return qml.probs(wires=data_wires)

In [49]:

for i in range(4):
    print(bin(np.argmax(QRAM([0, 0, 1, 1], i))))


0b11
0b11
0b11
0b11


In [42]:


# for x in range(0, 2**(2**N)):
#     print("x = ", x)
#     print("QRAM(x) = ", QRAM(x, 0))
#     print("QRAM(x) = ", bin(np.argmax(QRAM(x, 0))))
#     print("QRAM(x) = ", np.argmax(QRAM(x, 0)))
# print(bin(np.argmax(QRAM(x))))
# print(QRAM(x))